In [ ]:
import Ipynb_importer
from server_socket import ServerSocket
from socket_wrapper import SocketWrapper
from threading import Thread
from response_protocol import ResponseProtocol
from config import *
import os
import time

class Server(object):  # 服务器核心
    def __init__(self):
        self.server_socket = ServerSocket()  # 创建服务器套接字
        
        # 存不同请求对应的函数的字典
        self.request_handle_function = {}
        self.register(REQUEST_LOGIN, self.request_login_handle)
        self.register(REQUEST_CHAT, self.request_chat_handle)
        self.register(REQUEST_TIME, self.request_time_handle)
        self.register(REQUEST_FILE, self.request_file_handle)
        self.register(REQUEST_EXIT, self.request_exit_handle)
        
        self.clients = {}  # 保存当前在线用户的字典
        
    def register(self, request_id, handle_function):  # 注册消息与处理函数到字典中
        self.request_handle_function[request_id] = handle_function
        
    def startup(self):  # 获取客户端连接并提供服务
        while True:
            # 获取客户端连接
            print("Waiting for connection ...")
            soc, addr = self.server_socket.accept()
            print("Connected to the client sucessfully.")

            # 收发消息
            client_soc = SocketWrapper(soc)
            
            # 开启多线程: 一般写法
            # thread = Thread(target = self.request_handle, args = (client_soc, ))  # 注意只有一个元素的元组也要加,
            # thread.start()
            
            # 开启多线程: Lambda函数写法
            Thread(target = lambda: self.request_handle(client_soc, )).start()

            # soc.close()  # 主线程不关闭客户端套接字
    
    def request_handle(self, client_soc):  # 处理客户端请求
        while True:
                # 接收客户端消息
                message = client_soc.recv_data()
                if message == "/logout": break
                # print(message)
                # client_soc.send_data("The server got " + message)  # 向客户端发送提示信息
                
                # 解析数据
                parse_data = self.parse_request_text(message)
                
                # 分析请求类型并处理
                # print("parse_data: %s" % parse_data)
                
                # handle_function = self.request_handle_function[parse_data["request_id"]]  # 这样写key不存在时会报错
                handle_function = self.request_handle_function.get(parse_data["request_id"])
                if handle_function:  # 若存在该函数则调用
                    handle_function(client_soc, parse_data)
                
        self.remove_offline_user(client_soc)
        client_soc.close()  # 关闭客户端套接字
        
    def remove_offline_user(self, client_soc):  # 处理下线的客户端
        # print("Connection closed.")
        for username, info in self.clients.items():
            if info["sock"] == client_soc:
                del self.clients[username]
                print("Now online: ", self.clients)
                break
        
    def parse_request_text(self, text):  # 解析客户端发送的数据
        '''
        登录请求: 0001 | username | password
        聊天请求: 0002 | username | message
        获取服务器时间请求: 0003 | username
        获取服务器文件请求: 0004 | username | filename
        客户端与服务器断开连接请求: 0006 | username
        '''
        print("The client sent: " + text)
        
        # 按 | 分割消息
        request_list = text.split(DELIMITER)
        request_data = {}
        request_data["request_id"] = request_list[0]
        
        if request_data["request_id"] == REQUEST_LOGIN:  # 登录请求
            request_data["username"] = request_list[1]
            request_data["password"] = request_list[2]
        elif request_data["request_id"] == REQUEST_CHAT:  # 聊天请求
            request_data["username"] = request_list[1]
            request_data["message"] = request_list[2]
        elif request_data["request_id"] == REQUEST_TIME:  # 获取服务器时间请求
            request_data["username"] = request_list[1]
        elif request_data["request_id"] == REQUEST_FILE:  # 获取服务器文件请求
            request_data["username"] = request_list[1]
            request_data["filename"] = request_list[2]
        elif request_data["request_id"] == REQUEST_EXIT:  # 客户端与服务器断开连接请求
            request_data["username"] = request_list[1]
        return request_data
    
    def request_login_handle(self, client_soc, request_data):  # 处理登录请求
        # print("A login handle was got")
        
        # 获取账号、密码
        username = request_data["username"]
        password = request_data["password"]
        
        # 检查是否能登录
        result, nickname, username = self.check_user_login(username, password)
        if result == "1":  # 若登录成功则保存在线用户
            self.clients[username] = { "sock" : client_soc, "nickname" : nickname }
            print("Now online: ", self.clients)
            
        # 拼接消息并发给客户端
        response_text = ResponseProtocol.response_login_result(result, nickname, username)
        client_soc.send_data(response_text)
    
    def check_user_login(self, username, password):  # 检查用户能否登录成功, 若能则result = "1", 否则result = "0"
        if username not in accounts:
            return "0", username , ""
        elif password != accounts[username][0]:
            return "0", username , ""
        else:
            return "1", accounts[username][1], username
    
    def request_chat_handle(self, client_soc, request_data):  # 处理聊天请求
        # print("A chat handle was got.")
        
        # 获取消息内容
        username = request_data["username"]
        message = request_data["message"]
        nickname = self.clients[username]["nickname"]
        
        msg = ResponseProtocol.response_chat(nickname, message)  # 拼接发送给客户端的消息
        
        # 将消息转化给除发送者外的在线用户
        for _username, info in self.clients.items():
            if username != _username: 
                info["sock"].send_data(msg)
                
    def request_time_handle(self, client_soc, request_data):  # 处理获取服务器时间请求
        username = request_data["username"]
        msg = ResponseProtocol.response_time()
        for _username, info in self.clients.items():
            if username == _username:
                info["sock"].send_data(msg)
                break
                
    def request_exit_handle(self, client_soc, request_data):  # 处理客户端与服务器断开连接请求
        username = request_data["username"]
        msg = ResponseProtocol.response_exit()
        for _username, info in self.clients.items():
            if username == _username:
                info["sock"].send_data(msg)
                break
                
        self.remove_offline_user(client_soc)
        client_soc.close()  # 关闭客户端套接字
                
    def request_file_handle(self, client_soc, request_data):  # 处理获取服务器文件请求
        username = request_data["username"]
        # filename = request_data["filename"]  # 测试用
        filename = request_data["filename"][:-1]  # 去掉\n
        filepath = os.path.join(FILE_PATH, filename)  # 文件路径
        filesize = str(os.path.getsize(filepath))  # 文件大小 (Bytes)
        msg = ResponseProtocol.response_file(filename, filesize)
        
        print("A request for the file %s (%s Bytes) received." % (filename, filesize))
        
        # 先将文件大小发送给用户
        for _username, info in self.clients.items():
            if username == _username:
                info["sock"].send_data(msg)
                break
                         
        file_size = int(filesize)  # 文件大小
        send_size = 0  # 已发送大小
        file = open(filepath, "rb")  # 以二进制读的形式打开文件
        sending = True  # 文件发送flag
        
        # 分批发送文件
        while sending:
            time.sleep(0.2)  # 防止因发送过快而导致消息粘连
            data = None
            if send_size + SEND_CHUNK > file_size:  # 最后一批
                data = file.read(file_size - send_size)
                print("Sent a chunk of %d Bytes." % (file_size - send_size))
                sending = False
            else:  # 非最后一批
                data = file.read(SEND_CHUNK)
                send_size += SEND_CHUNK
                print("Sent a chunk of 64 Bytes.")
                
            data = str(data, encoding = "gbk")  # 防止中文乱码
            
            for _username, info in self.clients.items():
                if username == _username:
                    msg = ResponseProtocol.response_send(data)
                    print(msg)
                    info["sock"].send_data(msg)
                    break
        
        file.close()  # 注意关闭文件
        print("Sent the file %s (%s Bytes) sucessfully." % (filename, filesize))
        
if __name__ == "__main__":
    Server().startup()

Waiting for connection ...
Connected to the client sucessfully.
Waiting for connection ...
The client sent: 0001|user1|11451
The client sent: 0001|user1|114514
Now online:  {'user1': {'sock': <socket_wrapper.SocketWrapper object at 0x0000018D5E98DF10>, 'nickname': 'konbi'}}
Connected to the client sucessfully.
Waiting for connection ...
The client sent: 0001|admin|1919810
Now online:  {'user1': {'sock': <socket_wrapper.SocketWrapper object at 0x0000018D5E98DF10>, 'nickname': 'konbi'}, 'admin': {'sock': <socket_wrapper.SocketWrapper object at 0x0000018D5EB5E640>, 'nickname': 'Hytidel'}}
The client sent: 0002|user1|Good morning!

The client sent: 0002|admin|Hello!

The client sent: 0003|admin
The client sent: 0003|user1
The client sent: 0002|user1|可以写中文

The client sent: 0002|user1|可以写中文


The client sent: 0002|user1|可以写中文


The client sent: 0002|user1|可以写中文


The client sent: 0002|user1|可以写中文


The client sent: 0002|user1|可以写中文




In [3]:
s = "abc\n"
print(s)
print(s[:-1])
print("a")

abc

abc
a
